# Exploration API

In [1]:
import requests
from pprint import pprint
import time
import pandas as pd
import plotly.express as px
import streamlit as st
import plotly.graph_objects as go
import numpy as np

In [2]:
user = "Gotwiller"

In [3]:
df = pd.read_csv(f"../Data/{user}.csv")
max_timestamp = df['uts'].max()
max_timestamp

1764571939

## Liste des amis suivis

In [4]:
from pprint import pprint

api_key = "ea311d73665c24b237160f90bcb986ff"
artist = "Daft Punk"
method = "artist.getInfo"

url = f"http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user={user}&api_key={api_key}&format=json"
# Envoi d'une requête GET
response = requests.get(url)

# Vérification du statut de la réponse
if response.status_code == 200:
    # Affichage des données JSON
    data = response.json()
    for user in data["friends"]["user"]:
        print(user["name"])
else:
    print(f"Erreur : {response.status_code}")

Mehdinho16
anessrb
LightAkabane
Kallikall
Boborlegamorf
amoham16
Cassy1508
Ayamathers
drikoooooo
tzisteph
Kronyx67
jardinier38
clara_llz
Petah_Parkour
Grimkujow
redneck2_0
GunLock38
Alxstg
Charloops
Iandir
IcaRio_
Shironekii
Watch77


In [11]:
api_key = "ea311d73665c24b237160f90bcb986ff"
method = "user.getInfo"

url = f"http://ws.audioscrobbler.com/2.0/?method={method}&user={user}&api_key={api_key}&format=json"
# Envoi d'une requête GET
response = requests.get(url)

# Vérification du statut de la réponse
if response.status_code == 200:
    # Affichage des données JSON
    data = response.json()
    playcount = int(data["user"]["playcount"])
    pprint(playcount)
else:
    print(f"Erreur : {response.status_code}")

Erreur : 400


In [14]:
method = "user.getrecenttracks"
limit = 200 
timestamp_start = 1756731733
playcount = 105000

all_tracks = []
page = 1
total_pages = playcount / 200 + 1

while page <= total_pages:
    url = f"http://ws.audioscrobbler.com/2.0/?method={method}&limit={limit}&user={user}&page={page}&from={timestamp_start}&api_key={api_key}&format=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        tracks = data['recenttracks']['track']
        all_tracks.extend(tracks)
        
        # mise à jour du nombre total de pages
        total_pages = int(data['recenttracks']['@attr']['totalPages'])
        print(f"Page {page}/{total_pages} récupérée, total tracks: {len(all_tracks)}")
        
        page += 1
        time.sleep(0.2)  # pour ne pas spammer l'API
    else:
        print(f"Erreur à la page {page} : {response.status_code}")
        break

# Affichage final de tous les tracks récupérés
pprint(all_tracks)


Erreur à la page 1 : 400
[]


## Changement de format

In [13]:
import csv

# Supposons que tes données JSON soient dans une liste appelée all_tracks
# all_tracks = [...]  # tes données JSON récupérées

output_file = "lastfm_tracks_filtered.csv"

with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Écriture de l'en-tête
    writer.writerow(["uts", "utc_time", "artist", "artist_mbid", "album", "album_mbid", "track", "track_mbid"])
    
    for track in all_tracks:
        # Exclure les tracks en cours de lecture
        nowplaying = track.get('@attr', {}).get('nowplaying', 'false')
        if nowplaying.lower() == 'true':
            continue  # passer au track suivant
        
        # Certains tracks peuvent ne pas avoir de date
        date_info = track.get("date", {})
        uts = date_info.get("uts", "")
        utc_time = date_info.get("#text", "")
        
        artist_name = track["artist"].get("#text", "")
        artist_mbid = track["artist"].get("mbid", "")
        album_name = track["album"].get("#text", "")
        album_mbid = track["album"].get("mbid", "")
        track_name = track.get("name", "")
        track_mbid = track.get("mbid", "")
        
        writer.writerow([uts, utc_time, artist_name, artist_mbid, album_name, album_mbid, track_name, track_mbid])

print(f"CSV généré sans les morceaux en lecture : {output_file}")


CSV généré sans les morceaux en lecture : lastfm_tracks_filtered.csv


## Concatenation csv

In [ ]:
new_df = pd.read_csv("lastfm_tracks_filtered.csv")
old_df = pd.read_csv(f"../Data/{user}.csv")
combined_df = pd.concat([old_df, new_df]).drop_duplicates().reset_index(drop=True)
combined_df.to_csv(f"../Data/{user}.csv", index=False)

# Exploration Visus Solo

### Téléchargement des données 

In [ ]:
df = pd.read_csv("../data/Gautier.csv")

# 2. Aperçu rapide
df.head()        # premières lignes

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1756792590,"02 Sep 2025, 05:56",eyeto8,NaN,Boy Meets the World,NaN,Sound of Time Passing,NaN
1,1756792013,"02 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,The Shift,NaN
2,1756791598,"02 Sep 2025, 05:39",eyeto8,NaN,The Old Electronic Eldorado,NaN,Audentity,NaN
3,1756791295,"02 Sep 2025, 05:34",eyeto8,NaN,The Old Electronic Eldorado,NaN,Urgent,NaN
4,1756705616,"01 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,Lost Disco Thrills,NaN


### Ajout de Colonne supplémentaires

In [ ]:
# Convertir la colonne de temps
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# Ajouter des colonnes utiles
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week  # Numéro de semaine ISO

In [ ]:
df.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,year,hour,weekday,week
0,1756792590,2025-09-02 05:56:00,eyeto8,NaN,Boy Meets the World,NaN,Sound of Time Passing,NaN,2025-09-02,2025,5,Tuesday,36
1,1756792013,2025-09-02 05:46:00,eyeto8,NaN,The Old Electronic Eldorado,NaN,The Shift,NaN,2025-09-02,2025,5,Tuesday,36
2,1756791598,2025-09-02 05:39:00,eyeto8,NaN,The Old Electronic Eldorado,NaN,Audentity,NaN,2025-09-02,2025,5,Tuesday,36
3,1756791295,2025-09-02 05:34:00,eyeto8,NaN,The Old Electronic Eldorado,NaN,Urgent,NaN,2025-09-02,2025,5,Tuesday,36
4,1756705616,2025-09-01 05:46:00,eyeto8,NaN,The Old Electronic Eldorado,NaN,Lost Disco Thrills,NaN,2025-09-01,2025,5,Monday,36


In [ ]:
# --- Option : sélectionner une année spécifique ---
year_selected = 2025
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# --- Regrouper par semaine et jour ---
heatmap_data = df_year.groupby(['week', 'weekday']).size().reset_index(name='plays')

# Pivot pour créer la matrice
matrix = heatmap_data.pivot(index='weekday', columns='week', values='plays').reindex(jours_order)

# Remplacer les 0 par NaN pour laisser les cases vides
matrix = matrix.replace(0, np.nan)

# --- Étiquettes pour axes ---
semaines = [f"W{w}" for w in matrix.columns]
jours = matrix.index.tolist()

# --- Création de la heatmap ---
fig = px.imshow(
    matrix.values,
    x=semaines,
    y=jours,
    text_auto=True,
    color_continuous_scale='Turbo'
)

# --- Mettre le fond blanc pour les NaN ---
fig.update_traces(
    hovertemplate="%{y}, %{x}: %{z}<extra></extra>",
    zmin=0
)
fig.update_layout(
    title=f"Weekly activity per week - Year {year_selected}",
    xaxis_title="Week of the year",
    yaxis_title="Day of the week",
    plot_bgcolor='white',   # fond de la zone de tracé
    paper_bgcolor='white'   # fond autour du graphique
)

fig.show()

In [ ]:
fig = px.line(daily_counts, 
              x="date", 
              y="plays", 
              title="Écoutes par jour", 
              markers=True)

# Nombre maximum d'écoute d'un jour
max_plays = daily_counts["plays"].max()
line_y = max_plays * 1.1  # +5% de marge verticale

fig.add_hline(
    y=max_plays,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Top nombre d'écoute: {max_plays}",
    annotation_position="top left"
)

# Ajuster les limites du graphe pour laisser de l’espace au-dessus
fig.update_yaxes(range=[0, line_y])

fig.show()

In [ ]:

artist_counts = df["artist"].value_counts().reset_index()
artist_counts.columns = ["artist", "plays"]
fig = px.bar(
    artist_counts.head(10),
    x="plays",
    y="artist",
    orientation="h",
    title="Top 10 artistes écoutés",
    color="plays",
    color_continuous_scale="viridis"
)
fig.show()


In [ ]:
hourly = df.groupby("hour").size().reset_index(name="plays")
px.area(hourly, x="hour", y="plays", title="Répartition des écoutes par heure", markers=True)


# Exploration Visus Duo

In [ ]:
df_kronyx = pd.read_csv("../data/Justin.csv")
df_gotwiller = pd.read_csv("../data/Gautier.csv")

# 2. Aperçu rapide
df_kronyx.head()      

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1759297343,"01 Oct 2025, 05:42",HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821
1,1759297204,"01 Oct 2025, 05:40",Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN
2,1759297023,"01 Oct 2025, 05:37",Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a
3,1759296861,"01 Oct 2025, 05:34",Grace Power,NaN,Siren Song,NaN,Siren Song,NaN
4,1759263010,"30 Sep 2025, 20:10",Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f


In [ ]:
df_gotwiller.head()  

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1756792590,"02 Sep 2025, 05:56",eyeto8,NaN,Boy Meets the World,NaN,Sound of Time Passing,NaN
1,1756792013,"02 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,The Shift,NaN
2,1756791598,"02 Sep 2025, 05:39",eyeto8,NaN,The Old Electronic Eldorado,NaN,Audentity,NaN
3,1756791295,"02 Sep 2025, 05:34",eyeto8,NaN,The Old Electronic Eldorado,NaN,Urgent,NaN
4,1756705616,"01 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,Lost Disco Thrills,NaN


In [ ]:
for df in [df_kronyx, df_gotwiller]:
    df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")
    # Ajouter des colonnes utiles
    df["date"] = df["utc_time"].dt.date
    df["hour"] = df["utc_time"].dt.hour
    df["weekday"] = df["utc_time"].dt.day_name()
    df["minute"] = df["utc_time"].dt.minute

In [ ]:
df_kronyx.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,hour,weekday,minute
0,1759297343,2025-10-01 05:42:00,HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821,2025-10-01,5,Wednesday,42
1,1759297204,2025-10-01 05:40:00,Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN,2025-10-01,5,Wednesday,40
2,1759297023,2025-10-01 05:37:00,Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a,2025-10-01,5,Wednesday,37
3,1759296861,2025-10-01 05:34:00,Grace Power,NaN,Siren Song,NaN,Siren Song,NaN,2025-10-01,5,Wednesday,34
4,1759263010,2025-09-30 20:10:00,Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f,2025-09-30,20,Tuesday,10


## Visualisations

In [ ]:
# --- Charger les données ---
df1 = pd.read_csv("../data/Justin.csv")
df2 = pd.read_csv("../data/Gautier.csv")
time_min = max(df1["uts"].min(), df2["uts"].min())
df1["user"] = "justin"
df2["user"] = "gautier"

df = pd.concat([df1, df2], ignore_index=True)
df= df[df["uts"] > time_min]
# --- Convertir la colonne de temps ---
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# --- Colonnes utiles ---
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week


In [ ]:
time_min
#1739983023

1739983023

In [ ]:

# --- Sélecteur d'année ---
annees_disponibles = sorted(df["year"].unique())
year_selected = st.selectbox("Année à analyser", annees_disponibles)

# Filtrer l'année sélectionnée
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 1. Regrouper par semaine / jour / utilisateur
comp = (
    df_year.groupby(["week", "weekday", "user"])
           .size()
           .reset_index(name="plays")
)

# 2. Pivot table → weekday × week × user
pivot = comp.pivot_table(
    index=["weekday", "week"],
    columns="user",
    values="plays",
    fill_value=0
).reset_index()

pivot.head()

# 3. Calcul de qui a le plus écouté
pivot["compare"] = pivot["justin"] - pivot["gautier"]

2025-11-21 11:25:47.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.744 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
pivot.head()

user,weekday,week,gautier,justin,compare,compare_flag
0,Friday,8,67.0,76.0,9.0,1
1,Friday,9,93.0,39.0,-54.0,-1
2,Friday,10,4.0,49.0,45.0,1
3,Friday,11,8.0,7.0,-1.0,-1
4,Friday,12,30.0,14.0,-16.0,-1


In [ ]:
# -1 : Gautier gagne / 0 : égalité / +1 : Justin gagne
pivot["compare_flag"] = pivot["compare"].apply(lambda x:
    1 if x > 0 else (-1 if x < 0 else 0)
)

# 4. Matrice pour la heatmap
matrix_compare = pivot.pivot(
    index="weekday",
    columns="week",
    values="compare_flag"
).reindex(jours_order)

# 5. Remap des valeurs en [0,1] pour Plotly
# -1 → 0 (jaune)
#  0 → 0.5 (gris)
# +1 → 1 (bleu)
matrix_plot = (matrix_compare + 1) / 2

# --- Labels axes ---
semaines = [f"W{w}" for w in matrix_compare.columns]
jours = matrix_compare.index.tolist()


fig = px.imshow(
    matrix_plot,
    x=semaines,
    y=jours,
    color_continuous_scale=["yellow", "lightgray", "blue"],
    aspect="auto",
)

fig.update_traces(
    hovertemplate="%{y}, %{x}<extra></extra>"
)

fig.update_layout(
    title=f"Qui écoute le plus ? — Année {year_selected}",
    xaxis_title="Semaine",
    yaxis_title="Jour de la semaine",
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()
st.plotly_chart(fig)


2025-11-21 11:32:05.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
df = pd.read_csv("../temp_duo.csv")
df = df[df["track"] == "Intro"]
df
df[df["artist"] == "Kazzey"].groupby("user").size()

user
Gautier    17
dtype: int64